In [ ]:
import os
cwd = os.getcwd()
print(f'Current working directory: {cwd}')

project_path = os.path.join(cwd, 'FrameFinderLE')
print(f'Project directory: {project_path}')
database_path = os.path.join(cwd, 'FrameFinderLE', 'database')
print(f'Database directory {database_path}')

## Install dependency

In [ ]:
%cd {project_path}

In [ ]:
!pip show python-multipart

In [ ]:
#uncomment and run this line if running on Colab:
#!pip -q install nest_asyncio pyngrok multiprocess
#!pip -q install fastapi uvicorn python-multipart

!pip -q install -r requirements.txt

## Load database

In [ ]:
import gdown

#Load images
#Load key frame folder (key_frame_folder_reduced.zip)
!gdown 1-92UIqmQ5ODeZlSQ61cjFmUdLVZZ_HfV

#Unzip key frame folder (key_frame_folder_reduced)
!unzip -q key_frame_folder_reduced.zip -d static/images

In [ ]:
%cd {database_path}

In [ ]:
#Load FAISS
#Load annotation (index_caption_hashtag_dict_v2.json)
!gdown 1-CDUlIAIYAk5L87tXlYFosbUXQQANam8
#Load database CLIP_v0 (merged_index_hnsw_baseline_v0.bin)
!gdown 1EvNEWTNPe8Tk20-Tn0O6BwAgURLJTHZP
#Load database CLIP_v2 (merged_index_hnsw_baseline_v2.bin)
!gdown 1-85d-oCWU39o9d8Ie0c5093fKTp0IpwO

#Load GRAPH
#Graph database (graph_data_full.pkl)
!gdown 1-AotePkVml3iQONPxCZeK-gDjQFI0Asb
#List of hashtag embeddings (hashtag_embeddings.pkl, hashtag_embeddings.bin)
!gdown 1ZRt1-qvJP2CJcWzGykWQVN9JLHBS5XFR
!gdown 1tZyr1h8yDJO_CXuMn5ounEEdiFKD530d

#Load encoded frames
!gdown 1-KQx8lD7tHJH-RpbLE9gUBA8k_VV5fsI

## App

In [ ]:
%cd {project_path}

In [ ]:
import os
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
from typing import Dict, Any
from models.model_init import load_model
from database.db_init import load_grafa_database, load_hashtag_embeddings, load_hashtag_embedding_bin
from database.db_init import load_annotation, load_encoded_frames, faiss_database_processing

#Creates a FastAPI instance
app = FastAPI()
# Initialize the model and its preprocessing function
device, model = load_model()
# Initialize the database
G, sparse_matrix, node_mapping, reverse_node_mapping = load_grafa_database()
hashtag_embeddings = load_hashtag_embeddings()
hashtag_embedding_index = load_hashtag_embedding_bin()
image_info_dict = load_annotation()
encoded_frames = load_encoded_frames(device)
clipv0_hnsw = faiss_database_processing('CLIP_v0')
clipv2_hnsw = faiss_database_processing('CLIP_v2')

# Initialize the shared database
app.state.device = device
app.state.model = model
app.state.G = G
app.state.sparse_matrix = sparse_matrix
app.state.node_mapping = node_mapping
app.state.reverse_node_mapping = reverse_node_mapping
app.state.hashtag_embeddings = hashtag_embeddings
app.state.hashtag_embedding_index = hashtag_embedding_index
app.state.image_info_dict = image_info_dict
app.state.encoded_frames = encoded_frames
app.state.clipv0_hnsw = clipv0_hnsw
app.state.FEEDBACK_STORE: Dict[str, Any] = {}
app.state.TEMP_FEEDBACK_STORE: Dict[str, Any] = {}

# Include routers
from routers.home_router import router as home_router
from routers.update_results_router import router as update_results_router
from routers.data_router import router as data_router
from routers.search_router import router as search_router
from routers.feedback_router import router as feedback_router
from routers.process_query_router import router as process_query_router

app.include_router(home_router)
app.include_router(update_results_router)
app.include_router(data_router)
app.include_router(search_router)
app.include_router(feedback_router)
app.include_router(process_query_router)

# Mount the content directory to serve static files
app.mount('/static/style',
          StaticFiles(directory=os.path.join(os.getcwd(), 'static/style')),
          name='style')
app.mount('/static/script',
          StaticFiles(directory=os.path.join(os.getcwd(), 'static/script')),
          name='script')
app.mount('/static/images/key_frame_folder_reduced',
          StaticFiles(directory=os.path.join(os.getcwd(), 'static/images/key_frame_folder_reduced')),
          name='key_frame_folder_reduced')

In [ ]:
#Run the APP on Colab Notebook
from pyngrok import ngrok
!ngrok authtoken 'Add your ngrok authentication'
import nest_asyncio
import uvicorn

# Create tunnel
public_url = ngrok.connect(8000)
# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()
# Run the FastAPI app using uvicorn
print(f'PLEAS CLICK THIS URL {public_url} TO ACCESS THE APP!!!')
uvicorn.run(app)

In [ ]:
# Kill tunnel
ngrok.disconnect(public_url=public_url)